In [ ]:
# !pip install streamlit
import os
from mlflow import log_metric, log_param, log_artifacts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import mlflow

pd.set_option("display.max_columns", 200)

mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment('FluShotLearning')

# Data importing

via Google Mount

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

# train_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Flu Shot Learning/data/training_set_features.csv", index_col="respondent_id")
# train_labels = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Flu Shot Learning/data/training_set_labels.csv", index_col="respondent_id")
# test_features = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Flu Shot Learning/data/test_set_features.csv", index_col="respondent_id")

via Uploading files

In [ ]:
train_features = pd.read_csv(r"C:\Development\FluShotLearning\data\training_set_features.csv", index_col="respondent_id",engine='python')
train_labels = pd.read_csv(r"C:\Development\FluShotLearning\data\training_set_labels.csv", index_col="respondent_id", engine='python')
test_features = pd.read_csv(r"C:\Development\FluShotLearning\data\test_set_features.csv", index_col="respondent_id", engine='python')

In [ ]:
train_features.head()

In [ ]:
train_features.shape

In [ ]:
train_features.dtypes

In [ ]:
train_labels.head()

In [ ]:
train_labels.shape

# EDA

In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)

n_obs = train_labels.shape[0]

(train_labels['h1n1_vaccine']
    .value_counts()
    .div(n_obs)
    .plot(kind='bar', stacked=True,title="Proportion of H1N1 Vaccine", color=['steelblue', 'red'],ax=ax[0]))

ax[0].set_ylabel("h1n1_vaccine")

(train_labels['seasonal_vaccine']
    .value_counts()
    .div(n_obs)
    .plot(kind='bar', stacked=True,title="Proportion of Seasonal Vaccine", color=['steelblue', 'red'],ax=ax[1]))

ax[1].set_ylabel("seasonal_vaccine")

fig.tight_layout()


In [ ]:
joined_df = train_features.join(train_labels)
print(joined_df.shape)
joined_df.head()

In [ ]:
def vaccination_rate_plot(col, target, data, ax=None):
    """Stacked bar chart of vaccination rate for `target` against 
    `col`. 
    
    Args:
        col (string): column name of feature variable
        target (string): column name of target variable
        data (pandas DataFrame): dataframe that contains columns 
            `col` and `target`
        ax (matplotlib axes object, optional): matplotlib axes 
            object to attach plot to
    """
    counts = (joined_df[[target, col]]
                  .groupby([target, col])
                  .size()
                  .unstack(target)
             )
    group_counts = counts.sum(axis='columns')
    props = counts.div(group_counts, axis='index')

    props.plot(kind="barh", stacked=True, ax=ax)
    ax.invert_yaxis()
    ax.legend().remove()



In [ ]:
cols_to_plot = train_features.columns.tolist()

fig, ax = plt.subplots(
    len(cols_to_plot), 2, figsize=(9,len(cols_to_plot)*2.5)
)
for idx, col in enumerate(cols_to_plot):
    vaccination_rate_plot(
        col, 'h1n1_vaccine', joined_df, ax=ax[idx, 0]
    )
    vaccination_rate_plot(
        col, 'seasonal_vaccine', joined_df, ax=ax[idx, 1]
    )
    
ax[0, 0].legend(
    loc='lower center', bbox_to_anchor=(0.5, 1.05), title='h1n1_vaccine'
)
ax[0, 1].legend(
    loc='lower center', bbox_to_anchor=(0.5, 1.05), title='seasonal_vaccine'
)
fig.tight_layout()

For correlation between features and variable we should use Cramer's V, or Theil's u

In [ ]:
def histograms_numeric_columns(df, numerical_columns):
    '''
    Takes df, numerical columns as list
    Returns a group of histagrams
    '''
    f = pd.melt(df, value_vars=numerical_columns) 
    g = sns.FacetGrid(f, col='variable',  col_wrap=4, sharex=False, sharey=False)
    g = g.map(sns.distplot, 'value')
    return g

def heatmap_numeric_w_dependent_variable(df, dependent_variable):
    '''
    Takes df, a dependant variable as str
    Returns a heatmap of all independent variables' correlations with dependent variable 
    '''
    plt.figure(figsize=(8, 10))
    g = sns.heatmap(df.corr()[[dependent_variable]].sort_values(by=dependent_variable), 
                    annot=True, 
                    cmap='coolwarm', 
                    vmin=-1,
                    vmax=1) 
    return g

def cramers_v(x, y):
    '''
    Returns cramers_v for 2 categorical features
    '''
    confusion_matrix = pd.crosstab(x,y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r,k = confusion_matrix.shape
    phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
    rcorr = r-((r-1)**2)/(n-1)
    kcorr = k-((k-1)**2)/(n-1)
    return round(np.sqrt(phi2corr/min((kcorr-1),(rcorr-1))),3)

def heatmap_categorical_columns_w_dependant_categorical(df, dependent_variable, columns):
    '''
    Takes df, a dependant variable as str
    Returns a heatmap of catecorical columns cramers_v with dependent variable 
    '''
    plt.figure(figsize=(8, 10))
    corrM = [cramers_v(df[dependent_variable], df[column]) for column in columns]
    corr = pd.DataFrame(corrM, index=columns, columns=[dependent_variable])
    ax = sns.heatmap(corr,
            annot=True,
            cmap='coolwarm', 
            vmin=-1,
            vmax=1,
           )
    ax.set_title("Cramer V Correlation between Variables")
    return ax

In [ ]:
joined_df.columns.shape

In [ ]:
categorical_cols = list(set(joined_df.columns.values.tolist()))
len(categorical_cols)

In [ ]:
heatmap_categorical_columns_w_dependant_categorical(joined_df, 'h1n1_vaccine' , categorical_cols)

In [ ]:
heatmap_categorical_columns_w_dependant_categorical(joined_df, 'seasonal_vaccine' , categorical_cols)

In [ ]:
corr = joined_df.corr()

corrM_h1n1 = [cramers_v(joined_df['h1n1_vaccine'], joined_df[column]) for column in categorical_cols]
corr_h1n1 = pd.DataFrame(corrM_h1n1, index=categorical_cols, columns=['h1n1_vaccine'])


corrM_seasonal = [cramers_v(joined_df['seasonal_vaccine'], joined_df[column]) for column in categorical_cols]
corr_seasonal = pd.DataFrame(corrM_seasonal, index=categorical_cols, columns=['seasonal_vaccine'])

In [ ]:
print(len(corr_h1n1))
print(corr_h1n1)

plt.figure(figsize=(8, 10))
ax1 = sns.heatmap(corr_h1n1, linewidths=1,
            cmap='coolwarm', vmax=1, vmin=-1, square=True)
plt.show()

In [ ]:
print(len(corr_seasonal))
print(corr_seasonal)

plt.figure(figsize=(8, 10))
ax1 = sns.heatmap(corr_seasonal, linewidths=1,
            cmap='coolwarm', vmax=1, vmin=-1, square=True)
plt.show()

## Conclusion

It seems that in categorical data, when we observe a feature against a label, when there is **little to no difference between category classes** (e.g. h1n1 and sex) we can safely **assume there is low or zero correlation**.

**Should there be a divided approach for the two labels?** ;since there are features which can be presumed to have no correlation for one label, and some correlation for another label, e.g. include some features for predicting h1n1 vaccination probability whilst excluding the same for seasonal vaccination?

There is some correlation between labels for the same feature, suggesting that feature is universally accepted as a criteria for prediction, whether feature for one label has same distribution, or scaled/standardized distribution for the second label, e.g. *employment industry* and *employment occupation*

We should reduce number of features based on correlation to the labels appropriately and seperately, and after that do some prediction according to the label correlation or how h1n1 vaccination is related to the seasonal vaccination.


# Data Wrangling

## Outlier detection

In [ ]:
# # find unique elements per column
# unique_elements_per_column={i:train_features[i].unique() for i in train_features.columns}
# print(unique_elements_per_column)

# # create possible category combinations
# category_pairs=list(itertools.combinations(train_features.columns,len(train_features.columns)))
# #print(len(category_pairs[0]))

# # find all possible combinations by category pair
# all_possible_combinations=[list(itertools.product(unique_elements_per_column[i[0]],unique_elements_per_column[i[1]])) for i in category_pairs]
# all_possible_combinations
# # sum of all possible combinations by category pair
# sum_combinations=[len(i) for i in all_possible_combinations]
# df_out=pd.DataFrame(columns=['all_possible_combinations'],index=category_pairs,data=sum_combinations)

## Data cleaning

In [ ]:
# print("Before dropping nan's",train_features.shape)
# train_features.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
# print("After dropping nan's", train_features.shape)

In [ ]:
multiple_nan_rows_to_drop = 9
all_rows = train_features.shape[1] + train_labels.shape[1]

print(train_features.shape)
merged_df = pd.merge(train_features, train_labels, left_index=True, right_index=True)
merged_df = merged_df.dropna(thresh = all_rows - multiple_nan_rows_to_drop)

merged_df.shape

train_labels = merged_df[["h1n1_vaccine", "seasonal_vaccine"]]
train_features = merged_df.drop(columns=["h1n1_vaccine", "seasonal_vaccine"])
print(train_features.shape)

Since there is only **approximately 1/4 of the data left** when **dropping missing values**, we should devise some different strategy to handling missing values, perhaps assume **predictions** based on **correlation between features**, or **replacing them with most common values** (mean, median, mode) but that has some drawbacks in bias. Maybe we should implement a strategy where the distribution is kept in place while using the best prediction method based on feature correlation.

Data for some features are label encoded, they need to be one-hot encoded or target encoded, see https://towardsdatascience.com/handling-categorical-data-the-right-way-9d1279956fc6



In [ ]:
threshold = 0.00

In [ ]:
h1n1_vaccine_columns_to_keep = corr_h1n1.loc[abs(corr_h1n1['h1n1_vaccine']) >= threshold].index.tolist()
h1n1_vaccine_columns_to_keep.remove('h1n1_vaccine')
len(h1n1_vaccine_columns_to_keep)

In [ ]:
seasonal_vaccine_columns_to_keep = corr_seasonal.loc[abs(corr_seasonal['seasonal_vaccine']) >= threshold].index.tolist()
seasonal_vaccine_columns_to_keep.remove('seasonal_vaccine')
len(seasonal_vaccine_columns_to_keep)

If the chosen approach is to **predict the labels together** with same columns, the columns to keep should be intersected

In [ ]:
h1n1_vaccine_columns_to_keep = set(h1n1_vaccine_columns_to_keep)
seasonal_vaccine_columns_to_keep = set(seasonal_vaccine_columns_to_keep)

intersecting_columns = list(h1n1_vaccine_columns_to_keep.union(seasonal_vaccine_columns_to_keep))
intersecting_columns.remove('h1n1_vaccine')
intersecting_columns.remove('seasonal_vaccine')
print(intersecting_columns)
print(len(intersecting_columns))

In [ ]:
train_features = train_features[intersecting_columns]
test_features = test_features[intersecting_columns]
train_features.shape

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.compose import ColumnTransformer

from sklearn.multioutput import MultiOutputClassifier

from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_curve, roc_auc_score

RANDOM_SEED = 6    # Set a random seed for reproducibility!

## Missing data

In [ ]:
missing_values_df = []

train_features.isnull().sum().sort_values(ascending=False)

In [ ]:
for col in train_features:
  print(col, " ", train_features[col].unique())

Replacing employee industry and occupation with some other value if they are not employed, other than NaN

In [ ]:
if "employment_status" in train_features.columns.tolist():
  employment_status = train_features[train_features["employment_status"] == 'Unemployed']
  employment_status = employment_status[['employment_status','employment_industry','employment_occupation']]
  for col in employment_status:
    print(col, " ", employment_status[col].unique())
  print(employment_status.shape)

In [ ]:
if "employment_status" in train_features.columns.tolist():
  if "employment_industry" in train_features.columns.tolist():
    train_features.loc[train_features['employment_status'] == 'Unemployed', 'employment_industry'] = 'unemployed'
    test_features.loc[test_features['employment_status'] == 'Unemployed', 'employment_industry'] = 'unemployed'
    
    train_features.loc[train_features['employment_status'] == 'Not in Labor Force', 'employment_industry'] = 'not_in_labor_force'
    test_features.loc[test_features['employment_status'] == 'Not in Labor Force', 'employment_industry'] = 'not_in_labor_force'

  if "employment_occupation" in train_features.columns.tolist():
    train_features.loc[train_features['employment_status'] == 'Unemployed', 'employment_occupation'] = 'unemployed'
    test_features.loc[test_features['employment_status'] == 'Unemployed', 'employment_occupation'] = 'unemployed'

    train_features.loc[train_features['employment_status'] == 'Not in Labor Force', 'employment_occupation'] = 'not_in_labor_force'
    test_features.loc[test_features['employment_status'] == 'Not in Labor Force', 'employment_occupation'] = 'not_in_labor_force'

Replacing Nan with 0 in health insurance, supposedly people didn't input health insurance because they don't have it

In [ ]:
train_features['health_insurance'] = train_features['health_insurance'].fillna(0)
test_features['health_insurance'] = test_features['health_insurance'].fillna(0)

train_features['income_poverty'] = train_features['income_poverty'].fillna('> $75,000')
test_features['income_poverty'] = test_features['income_poverty'].fillna('> $75,000')

train_features['doctor_recc_h1n1'] = train_features['doctor_recc_h1n1'].fillna(0)
test_features['doctor_recc_h1n1'] = test_features['doctor_recc_h1n1'].fillna(0)

train_features['rent_or_own'] = train_features['rent_or_own'].fillna('other')
test_features['rent_or_own'] = test_features['rent_or_own'].fillna('other')

# train_features['doctor_recc_seasonal'] = train_features['doctor_recc_seasonal'].fillna(1)
# test_features['doctor_recc_seasonal'] = test_features['doctor_recc_seasonal'].fillna(1)

train_features['employment_status'] = train_features['employment_status'].fillna('Unemployed')
test_features['employment_status'] = test_features['employment_status'].fillna('Unemployed')

# train_features['marital_status'] = train_features['marital_status'].fillna('Married')
# test_features['marital_status'] = test_features['marital_status'].fillna('Married')

# train_features['marital_status'] = train_features['marital_status'].fillna('other')
# test_features['marital_status'] = test_features['marital_status'].fillna('other')

# train_features['employment_status'] = train_features['employment_status'].fillna('Unemployed')
# test_features['employment_status'] = test_features['employment_status'].fillna('Unemployed')

In [ ]:
for col in train_features:
  print(col, " ", train_features[col].unique())

In [ ]:
# if "doctor_recc_h1n1" in train_features.columns.tolist():
#   employment_status = train_features[train_features["employment_status"].isna()]
#   employment_status = employment_status[['employment_status','age_group','education']]
#   for col in employment_status:
#     print(col, " ", employment_status[col].unique())
#   print(employment_status.shape)
  
#print(employment_status['health_insurance'].value_counts())

In [ ]:
train_features.isnull().sum().sort_values(ascending=False)

In [ ]:
# mode_train_features = {}
# mode_test_features = {}

# mode_train_features ['employment_occupation'] = train_features['employment_occupation'].value_counts().nlargest(10).iloc[[1]].index.values[0]
# mode_test_features ['employment_occupation']= test_features['employment_occupation'].value_counts().nlargest(10).iloc[[1]].index.values[0]

In [ ]:
# train_features['employment_occupation'].fillna(mode_train_features['employment_occupation'])
# test_features['employment_occupation'].fillna(mode_test_features['employment_occupation'])

In [ ]:
from sklearn.preprocessing import LabelEncoder

def unique_non_null(s):
    return s.dropna().unique()

#print(train_features.columns.tolist())
nominal_not_encoded_columns = ['hhs_geo_region','employment_status', 'employment_industry', 'employment_occupation', 'marital_status', 'rent_or_own', 'sex', 'census_msa', 'race']
nominal_two_var_not_encoded_columns = []
# encoding 2 value nominal variables into 0 and 1 instead of making 2 seperate variables in pd.get_dummies
label_encoder_nominal = LabelEncoder()
for col in nominal_not_encoded_columns:
  # Key Error handling
  if col in train_features.columns.tolist():
  #print(train_features[col].unique())
  #print(len(train_features[col].unique()))
    if(len(unique_non_null(train_features[col])) == 2):
      train_features[col] = label_encoder_nominal.fit_transform(train_features[col])
      test_features[col] = label_encoder_nominal.fit_transform(test_features[col])
      nominal_two_var_not_encoded_columns.append(col)
      print(train_features[col])

nominal_two_plus_var_not_encoded_columns = list(set(nominal_not_encoded_columns) - set(nominal_two_var_not_encoded_columns))
print(nominal_two_plus_var_not_encoded_columns)

# Key Error handling
nominal_two_plus_var_not_encoded_columns_intersection = set(nominal_two_plus_var_not_encoded_columns).intersection(set(train_features.columns.tolist()))

if(len(nominal_two_plus_var_not_encoded_columns_intersection) != 0):
  train_features = pd.get_dummies(train_features, prefix=nominal_two_plus_var_not_encoded_columns_intersection, columns=nominal_two_plus_var_not_encoded_columns_intersection)
  train_features.columns.tolist()
  test_features = pd.get_dummies(test_features, prefix=nominal_two_plus_var_not_encoded_columns_intersection, columns=nominal_two_plus_var_not_encoded_columns_intersection)
  train_features.columns.tolist()

# TO DO - label encode below column
ordinal_not_encoded_columns = ['education', 'age_group', 'income_poverty']
label_encoder = LabelEncoder()
for col in ordinal_not_encoded_columns:
  # Key Error handling
  if col in train_features.columns.tolist():
    train_features[col] = label_encoder.fit_transform(train_features[col])
    test_features[col] = label_encoder.fit_transform(test_features[col])
    print(train_features[col])

In [ ]:
ordinal_values = [1, 2, 3, 4, 5, np.nan]
ordinal_cols = []
for col in train_features:
  if train_features[col].isin(ordinal_values).all():
    ordinal_cols.append(col)

ordinal_cols.append('h1n1_concern')
ordinal_cols.append('h1n1_knowledge')
ordinal_cols.append('education')
ordinal_cols.append('income_poverty')
ordinal_cols.append('age_group')

print('Ordinal columns are :', ordinal_cols)


nominal_values = [0, 1, np.nan]
nominal_values_notencoded = [type(str), type(str), np.nan]
nominal_cols = []
for col in train_features:
  if (train_features[col].isin(nominal_values).all()) :
    nominal_cols.append(col)

print('Nominal columns are :', nominal_cols)

nominal_cols.append('hhs_geo_region')
nominal_cols.append('employment_industry')
nominal_cols.append('employment_occupation')
nominal_cols.append('employment_status')
nominal_cols.append('marital_status')
nominal_cols.append('rent_or_own')
nominal_cols.append('race')

categorical_cols = ordinal_cols + nominal_cols
categorical = list(set(categorical_cols).intersection(set(train_features.columns.tolist())))
print(categorical_cols)

numerical_cols = list(set(train_features.columns.tolist()) - set(categorical_cols))
#numerical_cols = train_features.columns.tolist() - categorical_cols
print(numerical_cols)


In [ ]:
for col in train_features:
  print(col, " ", train_features[col].unique())

# Feature engineering

In [ ]:
# seas_overall_opinion = [i for i in train_features.columns.tolist() if 'seas' in i]
# print(seas_overall_opinion)

# train_features["seas_overall_opinion"] = train_features[seas_overall_opinion].prod(axis=1) #sum prod
# test_features["seas_overall_opinion"] = test_features[seas_overall_opinion].prod(axis=1) 
# categorical_cols.append("seas_overall_opinion")

In [ ]:
# h1n1_overall_opinion = [i for i in train_features.columns.tolist() if 'h1n1' in i]
# print(h1n1_overall_opinion)

# train_features["h1n1_overall_opinion"] = train_features[h1n1_overall_opinion].prod(axis=1) #sum
# test_features["h1n1_overall_opinion"] = test_features[h1n1_overall_opinion].prod(axis=1) #sum
# categorical_cols.append("h1n1_overall_opinion")

In [ ]:
#median_cols = []
median_cols = ['opinion_seas_sick_from_vacc','opinion_seas_risk','income_poverty', 'opinion_h1n1_sick_from_vacc', 'h1n1_knowledge', 'h1n1_concern', 'opinion_seas_vacc_effective','household_children', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'education']
# 'opinion_h1n1_sick_from_vacc', 'h1n1_knowledge', 'h1n1_concern', 'opinion_seas_vacc_effective','household_children', 'opinion_h1n1_vacc_effective', 'age_group','opinion_h1n1_risk', 'education'
#'age_group',

for col in median_cols:
  train_features[f"{col}_median"] = (train_features[col] > train_features[col].median()).astype(int)
  test_features[f"{col}_median"] = (test_features[col] > test_features[col].median()).astype(int)
  categorical_cols.append(f"{col}_median")

mean_cols = []
# 'household_children', 'household_adults', 'age_group', 'household_children'

for col in mean_cols:
  train_features[f"{col}_mean"] = (train_features[col] > train_features[col].mean()).astype(int)
  test_features[f"{col}_mean"] = (test_features[col] > test_features[col].mean()).astype(int)
  categorical_cols.append(f"{col}_mean")


In [ ]:
# behavioral_cols = [i for i in train_features.columns.tolist() if i.startswith('behavioral')]
# print(behavioral_cols)

# train_features["behavioral"] = train_features[behavioral_cols].sum(axis=1) #sum prod
# test_features["behavioral"] = test_features[behavioral_cols].sum(axis=1)
# categorical_cols.append("behavioral")

In [ ]:
# train_features["behavioral"].unique()

In [ ]:
categorical_cols

In [ ]:
numerical_cols

In [ ]:
print(categorical_cols)
print(numerical_cols)
print(len(categorical_cols + numerical_cols))
print(len(train_features.columns.tolist()))

In [ ]:
categorical_cols = list(set(categorical_cols).intersection(set(train_features.columns.tolist())))
numerical_cols = list(set(numerical_cols).intersection(set(train_features.columns.tolist())))

In [ ]:
print(categorical_cols)
print(numerical_cols)
print(len(categorical_cols + numerical_cols))
print(len(train_features.columns.tolist()))

In [ ]:
print(len(categorical_cols + numerical_cols))

# Model 

In [ ]:
train_features

In [ ]:
# categorical_cols.append('household_children')
# categorical_cols.append('house_h')
cols_to_remove_or_append = ['h1n1_knowledge']


categorical_mode_cols = []
for element in cols_to_remove_or_append:
    categorical_cols.remove(element)
    categorical_mode_cols.append(element)

categorical_mode_cols

In [ ]:
substitute_cols = []
for col in categorical_mode_cols:
  substitute_cols.append(train_features.columns.get_loc(col))

categorical_mode_cols = list(substitute_cols)

substitute_cols = []
for col in categorical_cols:
  substitute_cols.append(train_features.columns.get_loc(col))

categorical_cols = substitute_cols

substitute_cols = []
for col in numerical_cols:
  substitute_cols.append(train_features.columns.get_loc(col))

numerical_cols = substitute_cols

In [ ]:
# chain preprocessing into a Pipeline object
# each step is a tuple of (name you chose, sklearn transformer)
numeric_preprocessing_steps = Pipeline([
    ('median_imputer_numerical', SimpleImputer(strategy='median')), ##### impute first
    ('standard_scaler', MinMaxScaler())
])

n_neighbors = 51

categorical_mode_preprocessing_steps = Pipeline([
    #('one_hot_encoding', OneHotEncoder(handle_unknown="ignore"))
    ('mode_imputer_categorical', SimpleImputer(strategy='most_frequent')),
    ('min_max_scaler', MinMaxScaler())
]) 

categorical_preprocessing_steps = Pipeline([
    #('one_hot_encoding', OneHotEncoder(handle_unknown="ignore"))
    ('knn_imputer_categorical', KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors)),
    ('min_max_scaler', MinMaxScaler())
])


# create the preprocessor stage of final pipeline
# each entry in the transformer list is a tuple of
# (name you choose, sklearn transformer, list of columns)
preprocessor = ColumnTransformer(
    transformers = [
        ('categorical_mode', categorical_mode_preprocessing_steps, categorical_mode_cols),
        ("categorical", categorical_preprocessing_steps, categorical_cols),
        ("numeric", numeric_preprocessing_steps, numerical_cols)
    ],
    remainder = "drop"
)

In [ ]:
%pip install xgboost

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
import xgboost as xgb

# xgb.XGBClassifier(verbosity=3)

## Select model



In [ ]:
estimator = MultiOutputClassifier(
    estimator = xgb.XGBClassifier(verbosity=0)
)

In [ ]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("clf", estimator),
])

In [ ]:
full_pipeline

When choosing params in CV, key must be present in below dictionary

In [ ]:
full_pipeline.get_params().keys()


## Train-test split

In [ ]:
test_size=0.65

X_train, X_eval, y_train, y_eval = train_test_split(
    train_features,
    train_labels,
    test_size=test_size,
    shuffle=True,
    stratify=train_labels,
    random_state=RANDOM_SEED
)

# Hyperparameter optimization

In [ ]:
def centerlin(center, step, nosteps, dtype):
    minus = np.linspace(center,center - step * nosteps , nosteps +1 , dtype = dtype)
    #print(minus)
    plus = np.linspace(center, center + step * nosteps, nosteps +1, dtype = dtype)
    #print(plus)
    listnumbers = list(set(minus).union(set(plus)))
    listnumbers.sort()
    return listnumbers
    

def centerlog(center, step, nosteps, dtype):
    minus = np.logspace(center,center - step * nosteps , nosteps +1 , dtype = dtype)
    #print(minus)
    plus = np.logspace(center, center + step * nosteps, nosteps +1, dtype = dtype)
    #print(plus)
    listnumbers = list(set(minus).union(set(plus)))
    listnumbers.sort()
    return listnumbers

In [ ]:
params_RFC = {
    # 'clf__estimator__n_estimators': list(np.linspace(50, 4000, int((4000-50)/500) + 1, dtype=int)),
    'clf__estimator__max_depth': list(np.linspace(1,300, 15, dtype=int)),
    'clf__estimator__min_samples_split': list(np.linspace(2,300, 15, dtype=int)),
    'clf__estimator__min_samples_leaf': list(np.linspace(1,250, 15, dtype=int)),
    'clf__estimator__bootstrap': [True, False],
    'clf__estimator__criterion': ['gini', 'entropy']
}

params_LR = {
    'clf__estimator__C': np.logspace(-12, 12, 50),
    'clf__estimator__penalty': ['l1', 'l2'],
}

params_KNN = {
    'clf__estimator__leaf_size': list(np.linspace(1,100, 10, dtype=int)),
    'clf__estimator__n_neighbors': list(filter(lambda x: x%2 != 0, list(range(20,60)))),
    #'clf__estimator__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'clf__estimator__p': [1, 2, 3, 4]                                
}

params_SVC = {
    'clf__estimator__C': list(np.logspace(-2, 2, 5)),
    'clf__estimator__gamma': list(np.logspace(-2, 2, 5)),
    'clf__estimator__kernel': ['linear', 'sigmoid']
}

params_GNB = {
    'clf__estimator__var_smoothing': list(np.logspace(-2, 2, 50)),
}

params_GB = {
    'clf__estimator__learning_rate': list(np.logspace(-5,1, 5)),
    'clf__estimator__n_estimators': list(np.linspace(50, 500, 5, dtype=int)),
    'clf__estimator__max_depth': list(np.linspace(1, 32, 10, endpoint=True)),
    'clf__estimator__min_samples_split': list(np.linspace(0.1, 1.0, 10, endpoint=True)),
    'clf__estimator__min_samples_leaf':list(np.linspace(0.1, 0.5, 5, endpoint=True)),
    'clf__estimator__max_features':list(np.linspace(1,train_features.shape[1], 5, dtype=int))
}

params_XGB = {
    "clf__estimator__learning_rate": list(np.logspace(-5,1, 20)),
    "clf__estimator__max_depth": list(np.linspace(1, 50, 10, endpoint=True,dtype=int)),
    "clf__estimator__min_child_weight": list(np.logspace(1, 3, 20, endpoint=True)),
    "clf__estimator__gamma": list(np.logspace(-3, 3, 20)),
    "clf__estimator__colsample_bytree": list(np.logspace(-3, 0, 20)),
    "clf__estimator__colsample_bylevel": list(np.logspace(-2, -1, 20)),
    "clf__estimator__colsample_bynode": list(np.logspace(-2, -1, 10)),
    "clf__estimator__max_delta_step": list(np.logspace(-1.2,-0.6,10)),
    'clf__estimator__reg_lambda': list(np.logspace(-1,5,20)),
    'clf__estimator__reg_alpha': list(np.logspace(0.5,1.5,15))
}

In [ ]:
def determine_params(pipeline):

  pipeline_estimator_class = pipeline.steps[1][1].estimator.__class__

  RFC = RandomForestClassifier().__class__
  LR = LogisticRegression().__class__
  KNN = KNeighborsClassifier().__class__
  SVM = svm.SVC().__class__
  GNB = GaussianNB().__class__
  GB = GradientBoostingClassifier().__class__
  XGB = xgb.XGBClassifier().__class__

  if pipeline_estimator_class == RFC :
    params = params_RFC
  elif pipeline_estimator_class == LR :
    params = params_LR
  elif pipeline_estimator_class == KNN :
    params = params_KNN
  elif pipeline_estimator_class == SVM :
    params = params_SVC
  elif pipeline_estimator_class == GNB :
    params = params_GNB
  elif pipeline_estimator_class == GB :
    params = params_GB
  elif pipeline_estimator_class == XGB :
    params = params_XGB
  
  return params

In [ ]:
#params = determine_params(full_pipeline)
#params

## Validation curves

In [ ]:
from sklearn.model_selection import validation_curve

def validation_curve_for_number_param(param_string, pipeline):
  param_range = params[param_string]

  #if not all(not isinstance(flag, str) for (flag) in param_range):
  #  return


  train_scores, test_scores = validation_curve(
      pipeline,
      X_train,
      y_train,
      param_name=param_string,
      param_range=param_range,
      scoring="roc_auc",
      n_jobs=-1,
      verbose=10
  )
  train_scores_mean = np.mean(train_scores, axis=1)
  train_scores_std = np.std(train_scores, axis=1)
  test_scores_mean = np.mean(test_scores, axis=1)
  test_scores_std = np.std(test_scores, axis=1)

  plt.title("Validation Curve with Logistic Regression")
  plt.xlabel(param_string)
  plt.ylabel("Score")
  plt.xticks(range(len(param_range)), param_range, rotation=45)
  plt.ylim(0.75, 1.0)

  lw = 2
  plt.semilogx(
      param_range, train_scores_mean, label="Training score", color="darkorange", lw=lw
  )
  plt.fill_between(
      param_range,
      train_scores_mean - train_scores_std,
      train_scores_mean + train_scores_std,
      alpha=0.2,
      color="darkorange",
      lw=lw,
  )
  plt.semilogx(
      param_range, test_scores_mean, label="Cross-validation score", color="navy", lw=lw
  )
  plt.fill_between(
      param_range,
      test_scores_mean - test_scores_std,
      test_scores_mean + test_scores_std,
      alpha=0.2,
      color="navy",
      lw=lw,
  )
  plt.legend(loc="best")
  plt.grid(True)
  plt.show()

In [ ]:
# for key in params:
#   validation_curve_for_number_param(key, full_pipeline)

In [ ]:
params_RFC = {
    # 'clf__estimator__n_estimators': list(np.linspace(50, 4000, int((4000-50)/500) + 1, dtype=int)),
    'clf__estimator__max_depth': list(np.linspace(1,300, 15, dtype=int)),
    'clf__estimator__min_samples_split': list(np.linspace(2,300, 15, dtype=int)),
    'clf__estimator__min_samples_leaf': list(np.linspace(1,250, 15, dtype=int)),
    'clf__estimator__bootstrap': [True, False],
    'clf__estimator__criterion': ['gini', 'entropy']
}

params_LR = {
    'clf__estimator__C': np.logspace(-12, 12, 50),
    'clf__estimator__penalty': ['l1', 'l2'],
}

params_KNN = {
    'clf__estimator__leaf_size': list(np.linspace(1,100, 10, dtype=int)),
    'clf__estimator__n_neighbors': list(filter(lambda x: x%2 != 0, list(range(20,60)))),
    #'clf__estimator__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'clf__estimator__p': [1, 2, 3, 4]                                
}

params_SVC = {
    'clf__estimator__C': list(np.logspace(-2, 2, 5)),
    'clf__estimator__gamma': list(np.logspace(-2, 2, 5)),
    'clf__estimator__kernel': ['linear', 'sigmoid']
}

params_GNB = {
    'clf__estimator__var_smoothing': list(np.logspace(-2, 2, 50)),
}

params_GB = {
    'clf__estimator__learning_rate': list(np.logspace(-5,1, 10)),
    'clf__estimator__n_estimators': list(np.linspace(50, 500, 5, dtype=int)),
    'clf__estimator__max_depth': list(np.linspace(1, 10, 10, endpoint=True)),
    'clf__estimator__min_samples_split': list(np.logspace(-3, 0, 10, endpoint=True)),
    'clf__estimator__min_samples_leaf':list(np.linspace(0.1, 0.5, 5, endpoint=True)),
    'clf__estimator__max_features':list(np.linspace(1,train_features.shape[1], 5, dtype=int))
}

params_XGB = {
    "clf__estimator__learning_rate": list(np.linspace(0.05,0.15, 20)),
    "clf__estimator__max_depth": list(np.linspace(1, 50, 10, endpoint=True,dtype=int)),
    # "clf__estimator__max_depth": list(np.concatenate((np.linspace(5, 9, 5, endpoint=True, dtype=int),(np.linspace(42, 46, 5, endpoint=True, dtype=int))))), # rollback to previous values 1,
    "clf__estimator__min_child_weight": list(np.linspace(12, 13, 20, endpoint=True)),
    "clf__estimator__gamma": list(np.logspace(-3, 3, 20)),
    "clf__estimator__colsample_bytree": list(np.logspace(-3, 0, 20)) 

    

    #"clf__estimator__learning_rate": list(np.logspace(-2,-1,10, endpoint=True)),
    #"clf__estimator__max_depth": list(np.linspace(1, 50, 6, endpoint=True,dtype=int)),
    #"clf__estimator__min_child_weight": list(np.logspace(1.5, 2.5, 10, endpoint=True)),
    #"clf__estimator__gamma": list(np.logspace(0.5, 1.5, 10)),
    #"clf__estimator__colsample_bytree": list(np.logspace(-1, 0, 5,endpoint=True)),
    #"clf__estimator__colsample_bylevel": list(np.append(np.logspace(-2, -1, 5),1)),
    #"clf__estimator__colsample_bynode": list(np.append(np.logspace(-2, -1, 5),1)),
    #"clf__estimator__max_delta_step": list(np.append(np.logspace(-1.2,-0.6,10),0)),
    #'clf__estimator__reg_lambda': list(np.append(np.logspace(1.5,3.25,10),1)),
    #'clf__estimator__reg_alpha': list(np.append(np.logspace(0.5,1.5,10),0))
}

params = determine_params(full_pipeline)
params

In [ ]:
params

## Cross-Validation

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
import time

# params = 

n_iter = 300
cv = 20

t0 = time.time()
print("Fitting started...")
search = RandomizedSearchCV(full_pipeline, param_distributions=params, verbose=10, n_iter = n_iter, error_score="raise", cv=cv) #error_score="raise"
search.fit(X_train, y_train)
print(f"Fitting took {time.time() - t0:0.3f}s.")

In [ ]:
train_best_params = search.best_params_

In [ ]:
# from sklearn.model_selection import GridSearchCV
# import time

# t0 = time.time()
# print("Fitting started...")
# search = GridSearchCV(full_pipeline, param_grid=params, verbose=10)
# search.fit(X_train, y_train)
# print(f"Fitting took {time.time() - t0:0.3f}s.")

In [ ]:
#search.best_params_

In [ ]:
preds = search.predict_proba(X_eval)
preds

In [ ]:
print("test_probas[0].shape", preds[0].shape)
print("test_probas[1].shape", preds[1].shape)

In [ ]:
y_preds = pd.DataFrame(
    {
        "h1n1_vaccine": preds[0][:, 1],
        "seasonal_vaccine": preds[1][:, 1],
    },
    index = y_eval.index
)
print("y_preds.shape:", y_preds.shape)
y_preds.head()

In [ ]:
def plot_roc(y_true, y_score, label_name, ax):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    ax.plot(fpr, tpr)
    ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
    ax.set_ylabel('TPR')
    ax.set_xlabel('FPR')
    ax.set_title(
        f"{label_name}: AUC = {roc_auc_score(y_true, y_score):.4f}"
    )

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(7, 3.5))

plot_roc(
    y_eval['h1n1_vaccine'], 
    y_preds['h1n1_vaccine'], 
    'h1n1_vaccine',
    ax=ax[0]
)
plot_roc(
    y_eval['seasonal_vaccine'], 
    y_preds['seasonal_vaccine'], 
    'seasonal_vaccine',
    ax=ax[1]
)
fig.tight_layout()

# Results

In [ ]:
roc_auc_score(y_eval, y_preds)

In [ ]:
%%time 

train_features = train_features.to_numpy()
train_labels = train_labels.to_numpy()

# params = {

search.fit(train_features, train_labels)

In [ ]:
valid_best_params = search.best_params_

In [ ]:
test_probas = search.predict_proba(test_features)
test_probas

## Submissions

In [ ]:
submission_df = pd.read_csv(r"C:\Development\FluShotLearning\data\submission_format.csv", index_col="respondent_id",engine='python')

In [ ]:
submission_df.head()

In [ ]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_features.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = test_probas[0][:, 1]
submission_df["seasonal_vaccine"] = test_probas[1][:, 1]

submission_df.head()

In [ ]:
timestr = time.strftime("%Y%m%d-%H%M%S")
submission_df.to_csv(f'{timestr}.csv', index=True)

In [ ]:
with mlflow.start_run():

  mlflow.log_param("estimator", full_pipeline.steps[1][1].estimator.__class__)
  mlflow.log_param("multiple_nan_rows_to_drop", multiple_nan_rows_to_drop)
  mlflow.log_param("threshold", threshold)
  mlflow.log_param("params", params)
  mlflow.log_param("KNN imputer", n_neighbors)
  mlflow.log_param("cv", cv)
  mlflow.log_param("n-iterations", n_iter)
  mlflow.log_param("train best params", train_best_params)
  mlflow.log_param("validation best params", valid_best_params)
  mlflow.log_param("test size", test_size)
  mlflow.log_metric("h1n1 score", roc_auc_score(y_eval['h1n1_vaccine'], y_preds['h1n1_vaccine']))
  mlflow.log_metric("seasonal score", roc_auc_score(y_eval['seasonal_vaccine'], y_preds['seasonal_vaccine']))     
  mlflow.log_metric("score", roc_auc_score(y_eval, y_preds))  
  mlflow.log_artifact(f'{timestr}.csv')            

  # tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

  # if tracking_url_type_store != "file":
  #     mlflow.sklearn.log_model(rf, "model", registered_model_name="RandomForestModel")
  # else:
  #     mlflow.sklearn.log_model(rf, "model") 